In [1]:

import pandas as pd


In [16]:

data_train_rc_df = pd.read_csv('train.csv', on_bad_lines='skip')
data_test_rc_df  = pd.read_csv('test.csv',  on_bad_lines='skip')


In [17]:

data_train_rc_df.rename(columns={'id': 'idx', 'text': 'sentence', 'target':'label'}, inplace=True)
data_test_rc_df.rename( columns={'id': 'idx', 'text': 'sentence'}, inplace=True)


In [18]:

data_train_rc_df


,idx,keyword,location,sentence,label
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [19]:

data_test_rc_df 


,idx,keyword,location,sentence
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [20]:

from datasets import load_dataset


In [21]:

from datasets import Dataset, DatasetDict


In [22]:

## datasets_rc = load_dataset("csv", data_files={"train": train_path_rc, "validation": validation_path_rc} )



In [23]:

# Create Dataset from DataFrame
train_dataset = Dataset.from_pandas( data_train_rc_df)
test_dataset  = Dataset.from_pandas( data_test_rc_df )


In [24]:

train_dataset


Dataset({
    features: ['idx', 'keyword', 'location', 'sentence', 'label'],
    num_rows: 7613
})

In [25]:

test_dataset


Dataset({
    features: ['idx', 'keyword', 'location', 'sentence'],
    num_rows: 3263
})

In [28]:

train_dataset[7]


{'idx': 13,
 'keyword': None,
 'location': None,
 'sentence': "I'm on top of the hill and I can see a fire in the woods...",
 'label': 1}

In [33]:

def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)


In [37]:

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}


In [34]:

task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16


In [38]:

sentence1_key, sentence2_key = task_to_keys[task]


In [39]:

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)


In [40]:

encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/7613 [00:00<?, ? examples/s]

In [41]:

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels =  2

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:

## metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
metric_name = "accuracy"

model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "no", ## "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=False,
    metric_for_best_model=metric_name
    ## push_to_hub=True,
)


In [50]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)


In [52]:

validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_dataset,
    ## eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/var/folders/q4/zdsjyw0d297_fn6_fh5n7g9h0000gn/T/ipykernel_3437/2514303391.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [54]:

import wandb
wandb.init(mode="disabled")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [55]:

trainer.train()


Step,Training Loss
500,0.430700
1000,0.322800
1500,0.251400
2000,0.201200


TrainOutput(global_step=2380, training_loss=0.2815425424014821, metrics={'train_runtime': 841.2127, 'train_samples_per_second': 45.25, 'train_steps_per_second': 2.829, 'total_flos': 532862788637772.0, 'train_loss': 0.2815425424014821, 'epoch': 5.0})